In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs
import subprocess
import seaborn as sns
from umap import UMAP
from yoda import draw
import ubergauss.optimization as uo 
import pandas as pd
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs

In [ ]:
import cmcomp
data = {}

# CMCOMPARE

In [ ]:
a,l = load_rfam(full= False, add_cov= False)
names = [aa.gf["AC"][3:] for aa in a]

# THIS IS HOW WE WOULD RERUN THE CALCULATION
# [cmcomp.dumpcm(name) for name in names]
# IMPORTANTRESULT = cmcomp.run_cmcompare_pairwise()
# IMPORTANTRESULT.to_csv('cmcompare_full_run')

# LOAD CMCOMP DATA 
cmcompare_data = pd.read_csv('cmcompare_full_run_2024_06_27')
cmcompare_data  = cmcompare_data.fillna(500) 
cmcompare_data['score1'] = cmcompare_data[['score', 'score2']].min(axis=1)
cmcompare_data['score3'] = cmcompare_data[['score', 'score2']].max(axis=1)





In [ ]:

# the df has similarities.. 
cmcompare_dist = cmcomp.to_dist(cmcomp.pivot_numpy(cmcompare_data))
data['cmcompare'] = cmcompare_dist
cmcompare_mAP = cmcomp.mAP(cmcompare_dist,l)
cmcompare_k_clans = cmcomp.k_clan_discovery(cmcompare_dist,l,50)

# YODA 

In [ ]:

import pandas as pd
from lmz import Range
from sklearn.metrics.pairwise import euclidean_distances
def atograph(ali):
    graph = ali2graph.writecons(ali)
    graph = ali2graph.dillute( graph, dilute1 = .7, dilute2 = .4 , fix_edges =True)
    # graph = ali2graph.conscut( graph,  consThresh= .95, replacelabel = False )
    graphs = ali2graph.set_weight( graph,  consThresh = .97 )
    # graphs = ali2graph.nearstem( graph, boost_range = 4, boost_thresh = .5, boost_weight = 1)
    graphs = ali2graph.donest( graph)
    return graph

graphs = ut.xxmap(atograph, a)
matrix = ygraphs.vectorize_graphs(graphs, normalization=True,
                      min_r = 2,min_d=1,d=3,
                      inner_normalization=True)


def mkdf(mat): 
    sim = mat @ mat.T
    # sim = -euclidean_distances(mat)
    data = [{'cm1':a, 'cm2':b , 'score1': sim[a,b]} for a in Range(l) for b in Range(l)]
    return pd.DataFrame(data)
    
yoda_data = mkdf(matrix)

In [ ]:
yoda_dist = cmcomp.to_dist(cmcomp.pivot_numpy(yoda_data))
yoda_mAP = cmcomp.mAP(yoda_dist,l)
yoda_k_clans = cmcomp.k_clan_discovery(yoda_dist,l,50)
data['yoda'] = yoda_dist
from sklearn.preprocessing import normalize
yoda_norm_dist = normalize(yoda_dist, axis=0)
yoda_norm_mAP = cmcomp.mAP(yoda_norm_dist,l)
yoda_norm_k_clans = cmcomp.k_clan_discovery(yoda_norm_dist,l,50)

# INFERNAL

In [ ]:
import inferencemodel as inf
SIM_INF = inf.readCmscanAndMakeTable((a,l), path = 'inftools/infernal.tbl')

infernal_dist = cmcomp.to_dist(SIM_INF)
data['infernal']=infernal_dist
indices = np.where(infernal_dist == 10000)
noise = np.random.normal(0, 1, size=len(indices[0]))
infernal_dist[indices] += noise

infernal_mAP = cmcomp.mAP(infernal_dist,l)
infernal_k_clans = cmcomp.k_clan_discovery(infernal_dist,l,50)


In [ ]:
# k clans plot 
# we also need to collect the results for the other normed methods -> 3 x 2 lines are in this plot 

r = []
for k,dist in data.items():
    for i,val in enumerate( cmcomp.k_clan_discovery(dist,l,50)[1:]):
        r+=[{'Distances':'raw','Method':k,'neighbors':i+1,'mean clan discovery':val}]
    
    dist_norm = normalize(dist, axis=0)

    for i,val in enumerate( cmcomp.k_clan_discovery(dist_norm,l,50)[1:]):
        r+=[{'Distances':'normalized','Method':k,'neighbors':i+1,'mean clan discovery':val}]
        
df = pd.DataFrame(r)


#sns.reset_defaults()
sns.set_theme()
sns.set_context("talk")
ax= sns.lineplot(df, x= 'neighbors', y= 'mean clan discovery',hue='Method', style='Distances') 
sns.move_legend( ax,"center left", bbox_to_anchor=(1, 0.5))
plt.show()
plt.close()

In [ ]:
# precision recall, there are also 3 methods and a hue that will be the normalisation, 
# we make a dataframe oO 
# we need to report mAP and AUC seperately i think

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

def get_pairwise_distances(dist_matrix, labels):
    pairwise_distances = []
    pairwise_labels = []
    n = dist_matrix.shape[0]

    for i in range(n):
        for j in range(i+1, n):
            pairwise_distances.append(dist_matrix[i][j])
            pairwise_labels.append(labels[i] == labels[j]) 

    return np.array(pairwise_distances), np.array(pairwise_labels)


def prec_recall_plot(distances, labels):
    distances, true_labels = get_pairwise_distances(distances, labels)
    precision, recall, thresholds = precision_recall_curve(true_labels, -distances) 
    print(f'{auc(recall, precision)=}')
    return precision, recall
    # Plot Precision-Recall curve




r = []
for k,dist in data.items():
    
    #for p,re in zip(*prec_recall_plot(dist,l)):
    #    r+=[{'Distances':'Raw','Method':k,'precision':p,'recall':re}]
    
    dist_norm = normalize(dist, axis=0)

    for p,re in zip(*prec_recall_plot(dist_norm,l)):
        r+=[{'Distances':'Normalized','Method':k,'precision':p,'recall':re}]
        
df = pd.DataFrame(r)


#sns.reset_defaults()
sns.set_theme()
sns.set_context("talk")
ax= sns.lineplot(df, y= 'precision', x= 'recall', hue='Method', style='Distances') 
sns.move_legend( ax,"center left", bbox_to_anchor=(1, 0.5))
plt.show()
plt.close()

In [ ]:


prec_recall_plot(DIST_INF,l)
plt.matshow(DIST_INF)
plt.show()